In [ ]:
# Image Ellipse Overlay Analysis
# This notebook reads rotation data and region properties to overlay ellipses on images

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os
from pathlib import Path

# Set up matplotlib for better display
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

## 1. Load and examine the data

# Load the CSV files
rotation_data = pd.read_csv('../data/null_datasets/MT19937/rotation_data.csv')
region_props = pd.read_csv('../data/null_datasets/MT19937/MT19937_region_props.csv')  # Assuming MT19937 based on your sample

print("Rotation Data:")
print(rotation_data.head())
print(f"\nRotation data shape: {rotation_data.shape}")

print("\nRegion Properties Data:")
print(region_props.head())
print(f"\nRegion props shape: {region_props.shape}")

## 2. Data preprocessing and matching

# Extract frame numbers from image names for matching
region_props['frame_number'] = region_props['image'].str.extract(r'rendition_(\d+)\.jpg').astype(int)

# Sort both dataframes by frame/index for easier matching
rotation_data_sorted = rotation_data.sort_values('KID_IDFrameKey').reset_index(drop=True)
region_props_sorted = region_props.sort_values('frame_number').reset_index(drop=True)

print(f"\nFrame numbers in region props: {sorted(region_props['frame_number'].unique())[:10]}...")
print(f"KID_IDFrameKey in rotation data: {sorted(rotation_data['KID_IDFrameKey'].unique())[:10]}...")

## 3. Define ellipse drawing function

def draw_ellipse_on_image(image_path, centroid_x, centroid_y, major_length, minor_length, 
                         orientation_radians, title="", save_path=None):
    """
    Draw an ellipse on an image based on region properties
    
    Parameters:
    - image_path: path to the image file
    - centroid_x, centroid_y: center of the ellipse
    - major_length, minor_length: axis lengths
    - orientation_radians: orientation in radians
    - title: plot title
    - save_path: optional path to save the result
    """
    try:
        # Load the image
        img = Image.open(image_path)
        img_array = np.array(img)
        
        # Create figure and axis
        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        
        # Display the image
        ax.imshow(img_array)
        
        # Convert orientation from radians to degrees for matplotlib
        orientation_degrees = np.degrees(orientation_radians)
        
        # Create ellipse patch
        ellipse = patches.Ellipse(
            (centroid_x, centroid_y),
            major_length,
            minor_length,
            angle=orientation_degrees,
            linewidth=2,
            edgecolor='red',
            facecolor='none',
            alpha=0.8
        )
        
        # Add ellipse to plot
        ax.add_patch(ellipse)
        
        # Mark the centroid
        ax.plot(centroid_x, centroid_y, 'ro', markersize=5, label='Centroid')
        
        # Set title and labels
        ax.set_title(title)
        ax.set_xlabel('X coordinate')
        ax.set_ylabel('Y coordinate')
        ax.legend()
        
        # Remove axis ticks for cleaner look
        ax.set_xticks([])
        ax.set_yticks([])
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"Saved: {save_path}")
        
        plt.show()
        
    except FileNotFoundError:
        print(f"Image not found: {image_path}")
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")

## 4. Process images with ellipse overlays

# Define the image directory (modify this path as needed)
image_directory = "images/"  # Change this to your actual image directory path

# Create output directory for processed images
output_dir = "output_images_with_ellipses/"
os.makedirs(output_dir, exist_ok=True)

print(f"Looking for images in: {image_directory}")
print(f"Output will be saved to: {output_dir}")

## 5. Process a subset of images (first 10 for demonstration)

n_images_to_process = min(10, len(region_props))

for i in range(n_images_to_process):
    row = region_props.iloc[i]
    
    # Construct image path
    image_path = os.path.join(image_directory, row['image'])
    
    # Get corresponding rotation data if available
    frame_num = row['frame_number']
    rotation_info = rotation_data[rotation_data['KID_IDFrameKey'] == frame_num]
    
    if not rotation_info.empty:
        obj_info = rotation_info.iloc[0]['Object']
        euler_info = f"Euler: ({rotation_info.iloc[0]['Euler_X']:.1f}, {rotation_info.iloc[0]['Euler_Y']:.1f}, {rotation_info.iloc[0]['Euler_Z']:.1f})"
        title = f"{row['image']} - {obj_info}\n{euler_info}\nArea: {row['area']}, Ratio: {row['axis_ratio_minor_to_major']:.3f}"
    else:
        title = f"{row['image']}\nArea: {row['area']}, Ratio: {row['axis_ratio_minor_to_major']:.3f}"
    
    # Define output path
    output_path = os.path.join(output_dir, f"ellipse_{row['image']}")
    
    # Draw ellipse on image
    draw_ellipse_on_image(
        image_path=image_path,
        centroid_x=row['centroid_x'],
        centroid_y=row['centroid_y'],
        major_length=row['axis_major_length'],
        minor_length=row['axis_minor_length'],
        orientation_radians=row['orientation_radians'],
        title=title,
        save_path=output_path
    )

## 6. Summary statistics and analysis

print("\n" + "="*50)
print("SUMMARY STATISTICS")
print("="*50)

print(f"\nTotal images in region props: {len(region_props)}")
print(f"Total frames in rotation data: {len(rotation_data)}")

print(f"\nArea statistics:")
print(f"  Mean area: {region_props['area'].mean():.2f}")
print(f"  Median area: {region_props['area'].median():.2f}")
print(f"  Min area: {region_props['area'].min()}")
print(f"  Max area: {region_props['area'].max()}")

print(f"\nAxis ratio statistics:")
print(f"  Mean ratio (minor/major): {region_props['axis_ratio_minor_to_major'].mean():.3f}")
print(f"  Median ratio: {region_props['axis_ratio_minor_to_major'].median():.3f}")

print(f"\nObject distribution in rotation data:")
if not rotation_data.empty:
    print(rotation_data['Object'].value_counts())

## 7. Create summary visualization

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Area distribution
axes[0, 0].hist(region_props['area'], bins=30, alpha=0.7, color='blue')
axes[0, 0].set_title('Distribution of Areas')
axes[0, 0].set_xlabel('Area (pixels)')
axes[0, 0].set_ylabel('Frequency')

# Axis ratio distribution
axes[0, 1].hist(region_props['axis_ratio_minor_to_major'], bins=30, alpha=0.7, color='green')
axes[0, 1].set_title('Distribution of Axis Ratios (Minor/Major)')
axes[0, 1].set_xlabel('Ratio')
axes[0, 1].set_ylabel('Frequency')

# Centroid positions
axes[1, 0].scatter(region_props['centroid_x'], region_props['centroid_y'], alpha=0.6, c=region_props['area'], cmap='viridis')
axes[1, 0].set_title('Centroid Positions (colored by area)')
axes[1, 0].set_xlabel('Centroid X')
axes[1, 0].set_ylabel('Centroid Y')
axes[1, 0].invert_yaxis()  # Invert y-axis to match image coordinates

# Orientation distribution
axes[1, 1].hist(region_props['orientation_radians'], bins=30, alpha=0.7, color='orange')
axes[1, 1].set_title('Distribution of Orientations')
axes[1, 1].set_xlabel('Orientation (radians)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 8. Instructions for use

print("\n" + "="*50)
print("INSTRUCTIONS FOR USE")
print("="*50)
print("""
1. Make sure your CSV files are named correctly:
   - 'rotation_data.csv' 
   - '{RNG_method}_region_props.csv' (e.g., 'MT19937_region_props.csv')

2. Update the 'image_directory' variable to point to your images folder

3. The script will:
   - Load both CSV files
   - Sample 10 random images and display them as subplots with ellipses
   - Show object type, area, and other properties in subplot titles
   - Save individual processed images to 'output_images_with_ellipses/' folder
   - Show summary statistics

4. To see new random samples:
   - Run: show_new_sample() for 10 new random images
   - Run: show_new_sample(n_samples=16) for 16 random images
   - Each call will show a different random selection

6. The subplot display automatically arranges images in a roughly square grid

7. The ellipse parameters are:
   - Center: (centroid_x, centroid_y)
   - Major axis length: axis_major_length
   - Minor axis length: axis_minor_length  
   - Orientation: orientation_radians
   - Color: Red outline, no fill
""")

FileNotFoundError: [Errno 2] No such file or directory: 'rotation_data.csv'